# Map Census Tracts
In this notebook, I use the development pipeline data of SF development in order to create an online map of residential construction by Census Tracts


In [36]:
#import packages
import pandas as pd
import numpy as np
import re as re
import json    # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs
import pprint  # library for making Python data structures readable
pp = pprint.PrettyPrinter()
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from geopy.distance import great_circle
# magic command to display matplotlib plots inline within the ipython notebook
%matplotlib inline

In [37]:
#specify file paths
import_path = "/Users/briangoggin/Dropbox/CP 255/SF Development Project/Intermediate Files/"
code_path = "/Users/briangoggin/Dropbox/CP 255/SF Development Project/Code/Maps/"

# Section 1: Create polygons with recently completed development

## Determine constructed units over time

First, I assume that units were constructed in the latest quarter for which the project had "construction" as a project status. I identify these observations.

In [38]:
full_df = pd.read_csv(import_path+"/pipeline.csv")

In [39]:
#create dataframes for line graph of construction, BP, and BI starts over time
cons_end = full_df[full_df['status'] == "CONSTRUCTION"].groupby(['lot_number'], as_index=False)['quarter_order'].max()
cons_end.rename(columns = {'quarter_order': 'consdate'}, inplace = True)
#merge data together to identify quarter that projects were built
full_df2 = full_df.merge(cons_end, on = 'lot_number', how = "outer")
full_df2 = full_df2[full_df2['consdate'] == full_df2['quarter_order']]
full_df2 = full_df2[full_df2['quarter_order'] != 18]

## Create Point Data

In [40]:
crs = {'init' :'epsg:4326'}
geometry = [Point(xy) for xy in zip(full_df2.lon, full_df2.lat)]
construction = GeoDataFrame(full_df2, crs=crs, geometry=geometry)
construction.crs = {'init' :'epsg:4326'}
#construction.plot();

## Import Zillow Neighborhood Boundaries

In [41]:
root = '/Users/briangoggin/Dropbox/CP 255/SF Development Project/Raw Data'
boundaries = gpd.read_file(root+'/SF Census Tracts/tl_2016_06_tract.shp')

In [42]:
#extract fips code
boundaries['fips'] = boundaries['GEOID'].str[0:5]
sf = boundaries[boundaries['fips'] =='06075']

In [43]:
#set boundaries into same geographic coordinate system as points
sf.crs = {'init' :'epsg:4326'}

## Combine Layers

In [44]:
#First, spatial join between points and neighborhood boundaries. Set 'how' to 'right' to preserve polygon geometries.
sfcum = gpd.sjoin(construction, sf, how = 'right', op='within')

In [45]:
#Next, dissolve by neighborhoods to get sum of units
sfcum['GEOID'] = sfcum['GEOID'].astype(int)
sfcum = sfcum[['NAMELSAD','GEOID', 'geometry', 'net_units', 'net_affordable_units']]
sf_map = sfcum.dissolve(by=['NAMELSAD', 'GEOID'], aggfunc='sum')

sf_map['net_units'].fillna(0, inplace = True)
sf_map['net_affordable_units'].fillna(0, inplace = True)

sf_map['net_units'] = sf_map['net_units'].astype(int)
sf_map['net_affordable_units'] = sf_map['net_affordable_units'].astype(int)

sf_map['index'] = sf_map.index
sf_map['name'] = sf_map['index'].astype(str).str.split(',').str[0].str.strip('(').str.replace("'", '')
sf_map['RegionID'] = sf_map['index'].astype(str).str.split(',').str[1].str.strip(')')
sf_map.drop('index', axis = 1, inplace = True)
#sf_map.head(20)

In [46]:
sf_map.head()

,,geometry,net_units,net_affordable_units,name,RegionID
NAMELSAD,GEOID,,,,,
Census Tract 101,6075010100,"POLYGON ((-122.421076 37.812889, -122.420182 3...",0,0,Census Tract 101,6075010100
Census Tract 102,6075010200,"POLYGON ((-122.426625 37.80974, -122.426527 37...",9,0,Census Tract 102,6075010200
Census Tract 103,6075010300,"POLYGON ((-122.418718 37.805932, -122.418381 3...",0,0,Census Tract 103,6075010300
Census Tract 104,6075010400,"POLYGON ((-122.414877 37.803542, -122.413718 3...",1,0,Census Tract 104,6075010400
Census Tract 105,6075010500,"POLYGON ((-122.405174 37.804763, -122.403495 3...",77,0,Census Tract 105,6075010500


In [49]:
sf_map['net_units'].describe(percentiles = [.05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95])
#sf_map[sf_map['net_units']<0]

count      197.000000
mean       139.426396
std        836.697621
min        -25.000000
5%           0.000000
10%          0.000000
20%          0.200000
30%          1.000000
40%          2.000000
50%          3.000000
60%          5.000000
70%          8.200000
80%         28.800000
90%        109.000000
95%        350.000000
max      10550.000000
Name: net_units, dtype: float64

In [50]:
#Define function to create categories for javascript maps. Each category will be separate dot color
def cats(value):
    
    if (value['net_units'] < 0): 
        field = 0
    
    elif (value['net_units'] >=0) & (value['net_units']<=30): 
        field = 1
        
    elif (value['net_units'] >=31) & (value['net_units']<=50):
        field = 2
        
    elif (value['net_units'] >=51) & (value['net_units']<=100):
        field = 3
        
    elif (value['net_units'] >=101) & (value['net_units']<=300):
        field = 4
    else: 
        field = 5
        
    return field


sf_map['unitcat'] = sf_map.apply(cats, axis = 1)

In [51]:
#export to geojson object
export_path = code_path
with open(export_path+'/Neighborhood Maps/sf_recent.js', 'w') as f:
    f.write('var dataset = {};'.format(sf_map.to_json()))

In [52]:
#export to csv for affordability analysis
export_path = import_path
sf_map.to_csv(import_path+'/completed_tracts.csv')

In [53]:
sf_map['net_units'].sum()

27467

# Section 2: Create polygons with currently proposed development only

In [ ]:
#isolate currently proposed development
current = full_df[full_df['quarter']=='Q32016']

In [ ]:
#create geodataframe for current development
crs = {'init' :'epsg:4326'}
geometry = [Point(xy) for xy in zip(current.lon, current.lat)]
current_geo = GeoDataFrame(current, crs=crs, geometry=geometry)

In [ ]:
#First, spatial join between points and neighborhood boundaries. Set 'how' to 'right' to preserve polygon geometries.
final_geo = gpd.sjoin(current_geo, boundaries, how = 'right', op='within')

In [ ]:
#Next, dissolve by neighborhoods to get sum of units
final_geo['REGIONID'] = final_geo['REGIONID'].astype(int)
final_geo = final_geo[['NAME','REGIONID', 'geometry', 'net_units', 'net_affordable_units']]
final_geo = final_geo.dissolve(by=['NAME', 'REGIONID'], aggfunc='sum')

final_geo['net_units'].fillna(0, inplace = True)
final_geo['net_affordable_units'].fillna(0, inplace = True)
final_geo['net_units'] = final_geo['net_units'].astype(int)
final_geo['net_affordable_units'] = final_geo['net_affordable_units'].astype(int)

final_geo['index'] = final_geo.index
final_geo['name'] = final_geo['index'].astype(str).str.split(',').str[0].str.strip('(').str.replace("'", '')
final_geo['RegionID'] = final_geo['index'].astype(str).str.split(',').str[1].str.strip(')')
final_geo.drop('index', axis = 1, inplace = True)

In [ ]:
#Define function to create categories for javascript maps. Each category will be separate dot color
def cats(value):
    if (value['net_units'] >=0) & (value['net_units']<=50): 
        field = 0
        
    elif (value['net_units'] >=51) & (value['net_units']<=200):
        field = 1
        
    elif (value['net_units'] >=201) & (value['net_units']<=500):
        field = 2
        
    elif (value['net_units'] >=501) & (value['net_units']<=2000):
        field = 3
    else: 
        field = 4
        
    return field


final_geo['unitcat'] = final_geo.apply(cats, axis = 1)

In [ ]:
#export to geojson object
export_path = code_path
with open(export_path+'/Neighborhood Maps/nb_current.js', 'w') as f:
    f.write('var dataset2 = {};'.format(final_geo.to_json()))